# Project 3 - Sequence models

See the Project 3 text for more information about what is done in this project. Also see PDF supplementing this assignment.

Import relevant modules:

In [3]:
import torch
from torch import nn, optim

torch.manual_seed(123)
torch.set_default_dtype(torch.double)

path = '../data/'